## Get the query results of "fmri & love" from PubMed:

In [5]:
from pathlib import Path
import requests

URL = "https://pubmed.ncbi.nlm.nih.gov/?term=%28%28love%5BTitle%2FAbstract%5D%29+AND+%28fmri%5BTitle%2FAbstract%5D%29%29+AND+%28%22Front+Psychol.%22%5BJournal%5D%29+NOT+%28Editorial%29&filter=datesearch.y_10"

# Set browser-like headers to avoid being blocked by the site
headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/125.0.0.0 Safari/537.36"
    ),
    "Accept-Language": "en-US,en;q=0.9",
    "Referer": "https://pubmed.ncbi.nlm.nih.gov/",
}

outfile = Path(
    f"front_psychol_fmri_love.html"
)

with requests.Session() as s:
    s.headers.update(headers)
    resp = s.get(URL, timeout=30)
    resp.raise_for_status()           # raise an error for non-200 responses
    # Use server-provided encoding when available; default to utf-8
    if not resp.encoding:
        resp.encoding = "utf-8"
    outfile.write_text(resp.text, encoding=resp.encoding)

print(f"Saved {outfile.resolve()} ({outfile.stat().st_size:,} bytes)")


Saved /Users/tren/Desktop/02/front_psychol_fmri_love.html (243,473 bytes)


# Extract PMIDs from a PubMed HTML page
Here we use **BeautifulSoup** and **regular expressions** to extract PMIDs from a saved PubMed HTML page.

In [9]:
from bs4 import BeautifulSoup
import re
from pathlib import Path
import pandas as pd

HTML_PATH = Path('front_psychol_fmri_love.html')  # change this if needed
html_text = HTML_PATH.read_text(encoding='utf-8', errors='ignore')

In [26]:
soup = BeautifulSoup(html_text, 'html.parser')
meta = soup.find('meta', attrs={'name': 'log_displayeduids'})
pmids_str=meta.get('content')
print(pmids_str)
pmids= re.findall(r'\d+', pmids_str)
# or simply pmids=pmids_str.split(',')
print(pmids)

26617535,27242579,32457675,32528365
['26617535', '27242579', '32457675', '32528365']


# Test LM Studio API

In [29]:
# See all the available models:
import json

url = "http://localhost:1234/v1/models"
response = requests.get(url)
models = response.json()
print(json.dumps(models, indent=4))

{
    "data": [
        {
            "id": "text-embedding-nomic-embed-text-v1.5",
            "object": "model",
            "owned_by": "organization_owner"
        },
        {
            "id": "meta-llama-3-8b-instruct",
            "object": "model",
            "owned_by": "organization_owner"
        },
        {
            "id": "gemma-3-12b-it-qat",
            "object": "model",
            "owned_by": "organization_owner"
        },
        {
            "id": "gemma-3-4b-it-qat",
            "object": "model",
            "owned_by": "organization_owner"
        },
        {
            "id": "qwen2.5-14b-instruct-mlx",
            "object": "model",
            "owned_by": "organization_owner"
        },
        {
            "id": "phi-3.5-mini-instruct",
            "object": "model",
            "owned_by": "organization_owner"
        },
        {
            "id": "llama-3.2-3b-instruct",
            "object": "model",
            "owned_by": "organization_owner"
 

In [31]:
# Test text -> text:

url = "http://localhost:1234/v1/chat/completions"

payload = {
    "model": "gemma-3-4b-it-qat",
    "messages": [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "1+1=?"}
    ]
}

response = requests.post(url, json=payload)
data = response.json()
print(data["choices"][0]["message"]["content"])

1 + 1 = 2



In [32]:
# Test [image, text] -> text:

import base64

with open("32528365_Table3.jpg", "rb") as f:
    image_base64 = base64.b64encode(f.read()).decode("utf-8")

payload = {
    "model": "gemma-3-4b-it-qat",
    "messages": [
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "What's this?"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{image_base64}"
                    }
                }
            ]
        }
    ]
}

response = requests.post(url, json=payload)
data = response.json()
print(data["choices"][0]["message"]["content"])

This table presents the results of a structural analysis, likely from a neuroimaging study (such as fMRI). Here's a breakdown of what the table shows:

**Overall Purpose:** The table identifies "significant clusters of activity for the main effects of video type." This means researchers were examining how brain regions responded differently to different types of video content (e.g., action vs. nature, or educational vs. entertainment).

**Columns Explained:**

*   **Structural location:** This column indicates the brain region where a significant difference in activity was found.  The locations are described using abbreviations (e.g., "Medial frontal pole," "Sup. temporal gyrus").
*   **Voxels:** This represents the number of individual brain measurement points (voxels) within that region showing a significant difference in activity based on the video type.  A higher number indicates a more robust finding for that region.
*   **Z-max:** This is the maximum Z-score within a cluster. The

# Write your own [X, Y, Z] coordinate extractor
You can either extract the coordinates from *.html or *.pdf of the 3 target articles:

https://pmc.ncbi.nlm.nih.gov/articles/PMC4863427
https://pmc.ncbi.nlm.nih.gov/articles/PMC7223160
https://pmc.ncbi.nlm.nih.gov/articles/PMC7264388

You can assume you know what tables to extract, IF NEEDED. 

The sprint goal is to generate info_data.csv BY YOURSELF.

In [37]:
import pandas as pd
df = pd.read_csv('info_data.csv')
print(df.head())
print(df['Keywords'][0])

       PMID    PMCID                                           Keywords  \
0  27242579  4863427  AI; MPFC; aMCC; fMRI; intrasexual competition;...   
1  27242579  4863427  AI; MPFC; aMCC; fMRI; intrasexual competition;...   
2  27242579  4863427  AI; MPFC; aMCC; fMRI; intrasexual competition;...   
3  27242579  4863427  AI; MPFC; aMCC; fMRI; intrasexual competition;...   
4  27242579  4863427  AI; MPFC; aMCC; fMRI; intrasexual competition;...   

   Table   X   Y   Z  
0      2   8  12  58  
1      2 -30  22   4  
2      2  42  10   0  
3      2  -6  14  42  
4      2 -62 -22  34  
AI; MPFC; aMCC; fMRI; intrasexual competition; pain empathy


In [38]:
# Work with AI to write your codes: